In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis
!pip install -U -q PyDrive
!pip install yfinance --upgrade --no-cache-dir

import pandas as pd
import re
import emoji
import nltk
import contractions
import torch

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras

import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

from google.colab import drive

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter

import datetime
from datetime import datetime, timedelta, date

import yfinance as yf
from datetime import datetime, timedelta, date

#mount's to Google Drive
drive.mount('/content/drive')

#connect's to Google Cloud SDK
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/combined_0.csv')
df = df.sample(frac=1)
df.head()

,symbol,message,datetime,user,message_id,Date,Time,label
200079,V,7 trades today dia ecte fb v amzn twtr,2014-05-07 20:17:48,168386,22743737,2014-05-07,20:17:48,1
83313,HD,eu spent everyday shorting hd 39 open guess co...,2019-05-22 13:49:23,543250,165056055,2019-05-22,13:49:23,0
37235,T,6 year low taking weekly pivots light volume b...,2018-07-25 18:45:13,984264,131357673,2018-07-25,18:45:13,0
78559,HD,home depot another dip buy seeking alpha hd,2019-11-22 02:11:01,2178417,184648816,2019-11-22,02:11:01,0
178321,V,v hammer 20 trend place,2018-08-16 02:01:15,1505519,134102862,2018-08-16,02:01:15,0


In [ ]:
df.drop(df[df['Date'] <= '2019-07-20'].index, inplace = True)
df['label'].value_counts()

1    27545
0    25205
Name: label, dtype: int64

Here we start the process with regression. 
1. Splitting
2. train and predict
3. Random forest

4. Report

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.10, random_state=42)

In [ ]:
y_train.value_counts()

1    24749
0    22726
Name: label, dtype: int64

In [ ]:
y_test.value_counts()

1    2796
0    2479
Name: label, dtype: int64

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

In [ ]:
tuned_parameters = {'vect__ngram_range': [(1, 2),(2, 2)], 'clf__C':[0.01,0.1,1], 'clf__penalty':["l1","l2"]}

text_clf = Pipeline([('vect', CountVectorizer()),('clf', LogisticRegression(max_iter=5000, solver='liblinear'))])

score = 'f1_macro'
logmodel_cv=GridSearchCV(text_clf,tuned_parameters,cv=5, scoring=score)
logmodel_cv.fit(x_train.values.astype('U'), y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf',
                                        LogisticRegression(max_iter=5000,
                                                           solver='liblinear'))]),
             param_grid={'clf__C': [0.01, 0.1, 1], 'clf__penalty': ['l1', 'l2'],
                         'vect__ngram_range': [(1, 2), (2, 2)]},
             scoring='f1_macro')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logmodel_cv.predict(x_test.values.astype('U')), digits=4))

              precision    recall  f1-score   support

           0     0.5514    0.5038    0.5266      2479
           1     0.5914    0.6366    0.6132      2796

    accuracy                         0.5742      5275
   macro avg     0.5714    0.5702    0.5699      5275
weighted avg     0.5726    0.5742    0.5725      5275



In [ ]:
logmodel_cv.best_params_

{'clf__C': 1, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 2)}

In [ ]:
logmodel_cv.best_estimator_

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(C=1, max_iter=5000, solver='liblinear'))])